In [1]:
!pip install -r requirements.txt
!conda install -c conda-forge librosa -y
# !pip install scipy
# !pip install opencv-python-headless

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.4
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [2]:
# Copy files over from S3
!aws s3 cp s3://video-wav2lip/video.mp4 sample_data/
!aws s3 cp s3://audio-wav2lip/audio.mp3 sample_data/
!ls sample_data

fatal error: An error occurred (404) when calling the HeadObject operation: Key "video.mp4" does not exist
download: s3://audio-wav2lip/audio.mp3 to sample_data/audio.mp3  
audio.mp3  image.jpeg	    lessonVid.mp4  polly.wav
elon.mp4   lessonAudio.mp3  polly.mp3


In [3]:
# Convert image to 3-second video (if user uploaded image instead)
!ffmpeg -loop 1 -i sample_data/image.jpeg -c:v libx264 -t 3 -pix_fmt yuv420p -vf scale=320:240 sample_data/video.mp4 -y

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9.3.0 (crosstool-NG 1.24.0.133_b0863d8_dirty)
  configuration: --prefix=/home/ec2-user/anaconda3/envs/JupyterSystemEnv --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1609680890771/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/home/conda/feedstock_root/build_artifacts/ffmpeg_1609680890771/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample 

In [4]:
!python inference.py --checkpoint_path checkpoints/wav2lip.pth --face "sample_data/video.mp4" --audio "sample_data/audio.mp3" --resize_factor 2

Using cpu for inference.
Reading video frames...
Number of frames available for inference: 75
Extracting raw audio...
ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9.3.0 (crosstool-NG 1.24.0.133_b0863d8_dirty)
  configuration: --prefix=/home/ec2-user/anaconda3/envs/JupyterSystemEnv --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1609680890771/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/home/conda/feedstock_root/build_artifacts/ffmpeg_1609680890771/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 

One option is to save the result video to our S3 bucket.

In [5]:
# !aws s3 cp results/result_voice.mp4 s3://wav2lip-results/result.mp4

Another is to send an email to the user with the result as a video attachment!

In [6]:
import os
from helpers.email_aws import send_email

send_email(is_lesson=False) if os.path.isfile("results/result_voice.mp4") else send_email(error=True)

EMAIL SENT


### Clean Up

1. Remove Wav2Lip generated video
2. Remove video and audio files
3. Remove email file
4. Remove user-uploaded face video from Amazon S3
5. Remove user-uploaded lesson video from Amazon S3

In [7]:
!rm results/result_voice.mp4

In [8]:
!rm sample_data/video.mp4
!rm sample_data/audio.mp3

In [9]:
!rm email.txt

In [10]:
!aws s3 rm s3://video-wav2lip/video.mp4 

delete: s3://video-wav2lip/video.mp4


In [11]:
!aws s3 rm s3://video-wav2lip/audio.mp3 

delete: s3://video-wav2lip/audio.mp3
